<a href="https://colab.research.google.com/github/LinYenShou/114-1/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai

In [2]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [4]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [20]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1-7xUMjl7-QQCpixs_bQT16qYVcjkj94VAYh7w1xdlZM/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

REQUIRED_COLUMNS = ["日期", "科目", "作業成績","平均","標準差"]

_auth_done = False
_gc = None
_ws = None

In [5]:
from datetime import datetime
import math # <--- 1. 導入 math 模組用於計算標準差

# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade_input = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade_input)
        except ValueError:
            print("❌ 成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        # 儲存的成績是一個列表 [日期, 科目, 分數]
        grades.append([today, subject, grade])
        print(f"✅ 已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

def analyze_grades(grades_data):
    """
    計算並輸出成績的平均值和標準差。
    """
    if not grades_data:
        return # 如果沒有成績，就停止計算

    # 2. 提取所有分數
    scores = [record[2] for record in grades_data]
    n = len(scores)

    # 3. 計算平均值 (Average)
    average = sum(scores) / n

    # 4. 計算標準差 (Standard Deviation)
    # 步驟 a: 計算每個分數與平均值的差的平方和
    variance_sum = sum([(score - average) ** 2 for score in scores])

    # 步驟 b: 計算變異數
    variance = variance_sum / n

    # 步驟 c: 開根號得到標準差
    std_dev = math.sqrt(variance)

    # 5. 輸出結果
    print("\n--- 成績統計分析 ---")
    print(f"總共輸入科目數： {n} 門")
    print(f"總分： {sum(scores)}")
    print(f"平均成績 (Average)： {average:.2f}")  # 保留兩位小數
    print(f"標準差 (Standard Deviation)： {std_dev:.2f}") # 保留兩位小數
    print("--------------------")

# --- 程式執行入口 ---
if __name__ == "__main__":
    all_grades = get_user_grades()

    # 在輸入完成後呼叫分析函式
    analyze_grades(all_grades)

    # 輸出所有原始記錄
    print("\n--- 原始輸入資料 ---")
    for record in all_grades:
        print(f"日期: {record[0]}, 科目: {record[1]}, 成績: {record[2]}")
    print("--------------------")

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-01, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：q

--- 成績統計分析 ---
總共輸入科目數： 1 門
總分： 10
平均成績 (Average)： 10.00
標準差 (Standard Deviation)： 0.00
--------------------

--- 原始輸入資料 ---
日期: 2025-10-01, 科目: a, 成績: 10
--------------------


In [16]:
def calculate_stats(grades_data):
    """
    計算成績的平均值和標準差。
    返回一個包含 (average, std_dev) 的元組。
    """
    if not grades_data:
        return 0, 0

    scores = [record[2] for record in grades_data]
    n = len(scores)

    average = sum(scores) / n

    variance_sum = sum([(score - average) ** 2 for score in scores])
    variance = variance_sum / n
    std_dev = math.sqrt(variance)

    return round(average, 2), round(std_dev, 2)

# --- 核心 AI 呼叫函式 (已修改) ---
def get_ai_summary_and_review(grades):
    """
    呼叫 Gemini 模型，整合所有成績與統計數據，並要求生成總結評語、表現分析和學習建議。
    """
    # 1. 確保你已經配置了 Gemini API Key
    # genai.configure(api_key="YOUR_API_KEY")
    model = genai.GenerativeModel('gemini-2.5-flash')

    # 2. 計算統計數據
    avg, std_dev = calculate_stats(grades)

    # 3. 準備給 AI 的提示：強調評語和分析的需求
    prompt_text = "你是一位資深教育顧問。請根據提供的學生成績和統計數據，生成一份詳細的報告。這份報告必須包含以下三個部分：\n\n"
    prompt_text += "1. **整體表現評語 (Review)：** 針對平均、科目數量和標準差，給予簡潔有力的總結評語。\n"
    prompt_text += "2. **表現分析與優勢/弱項 (Analysis)：** 分析成績的集中度（看標準差）和科目分數的分佈，指出可能的優勢領域和需要加強的弱項。\n"
    prompt_text += "3. **常見學習迷思與建議 (Advice)：** 提供針對此成績模式的常見學習迷思整理和實用建議。\n\n"

    prompt_text += "【統計數據】\n"
    prompt_text += f"總平均成績：{avg}\n"
    prompt_text += f"成績標準差：{std_dev}\n"
    prompt_text += f"總科目數：{len(grades)}\n\n"

    prompt_text += "【原始成績】\n"
    for record in grades:
        date, subject, grade = record
        prompt_text += f"日期：{date}, 科目：{subject}, 成績：{grade}\n"

    print("\n--- 正在呼叫 AI 模型生成報告... ---")
    try:
        response = model.generate_content(prompt_text)
        report = response.text
        return report
    except Exception as e:
        print(f"呼叫 AI 時發生錯誤：{e}")
        return "AI 報告生成失敗。"


In [10]:
new_grades = get_user_grades()

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-01, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
✅ 已記錄：日期: 2025-10-01, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q


In [11]:
new_grades

[['2025-10-01', 'a', 10], ['2025-10-01', 'b', 20]]

In [17]:
get_ai_summary_and_review(new_grades)


--- 正在呼叫 AI 模型生成報告... ---


"好的，這是一份根據您提供的學生成績和統計數據生成的詳細教育報告。\n\n---\n\n**【學術表現詳細評估報告】**\n\n**學生姓名：** [請填寫學生姓名]\n**評估日期：** 2025年10月01日\n**報告生成者：** 資深教育顧問\n\n---\n\n### **1. 整體表現評語 (Overall Performance Review)**\n\n從現有數據來看，該生目前僅有兩門科目的評估記錄，總平均成績為15.0分。標準差5.0分，這點出兩門科目之間的分數表現存在明顯的差異性。需強調的是，此評估僅基於極少數數據，不足以全面反映該生的真實學術潛力或全貌。\n\n### **2. 表現分析與優勢/弱項 (Performance Analysis & Strengths/Weaknesses)**\n\n*   **成績集中度與分佈：**\n    *   標準差5.0分在僅有兩門科目的情況下，直接反映了成績分佈的**兩極化**。這表示該生在這兩門科目上的表現並不均衡，存在一高一低的顯著差距，而非兩者分數相近。\n    *   具體來看，科目 'b' 以20分的表現，相較於科目 'a' 的10分，可視為該生目前相對的**優勢領域**。這說明在科目 'b' 上，學生可能具備較好的理解能力或投入程度。\n    *   而科目 'a' 的10分，則明顯是需要即刻關注和加強的**弱項**。該分數可能暗示對基礎概念的掌握不足，學習上存在較大的困難或障礙。\n\n*   **潛在優勢：**\n    *   科目 'b' 的相對高分（20分）顯示該生在特定學科領域存在一定的學習能力和潛力。儘管絕對分數不高，但它提供了一個比科目 'a' 更好的起點。這可能是該生學習興趣或天賦的初步顯現。\n\n*   **需加強的弱項：**\n    *   科目 'a' 的低分（10分）是一個明確的警示。這可能反映出幾個問題：\n        *   **基礎知識薄弱：** 缺乏對科目核心概念的理解。\n        *   **學習方法不適用：** 當前的學習策略可能不適合該科目的要求。\n        *   **學習興趣不足：** 對科目內容缺乏興趣，導致投入度不夠。\n        *   **外部干擾：** 可能有其他因素影響了對該科目的學習。\n   

In [22]:
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)





        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary_and_review(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()

--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：a
請輸入 a 的成績：10
✅ 已記錄：日期: 2025-10-01, 科目: a, 成績: 10

請輸入科目（或輸入 'q' 停止）：b
請輸入 b 的成績：20
✅ 已記錄：日期: 2025-10-01, 科目: b, 成績: 20

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---

--- 正在呼叫 AI 模型生成報告... ---

--- AI 摘要已成功寫入 Google Sheet。---
以下是 AI 生成的摘要內容：
--------------------------------------------------
## 教育顧問成績評估報告

**學生姓名：** [未提供]
**評估日期：** 2025年10月01日
**報告生成日期：** 2023年10月27日

---

### 1. 整體表現評語 (Review)

根據提供的有限數據，該學生的總平均成績為 **15.0分**，顯示整體學習表現處於**非常低迷**的水平。本次評估僅涵蓋了 **兩門科目**，這使得此平均數的代表性較為局限，難以全面反映學生的真實學習狀況或潛力。成績標準差為 **5.0分**，雖然科目數量少，但此數值仍指出這兩門科目之間存在**顯著的成績差異**。

### 2. 表現分析與優勢/弱項 (Analysis)

分析學生的兩門科目原始成績（科目a：10分，科目b：20分），可以觀察到以下特點：

*   **成績集中度與分佈：** 兩門科目分數分別為10分和20分，距離平均分15分各一個標準差。這不是成績集中度高的表現，反而是反映了即使科目數量少，學生在不同科目間的學習成果仍存在相當大的落差。科目b的得分是科目a的兩倍，這是一個極端的分佈，暗示了學習上的巨大不均衡。

*   **優勢領域（潛力發掘）：**
    *   **相對優勢：** 僅從現有數據來看，科目b的20分相較於科目a的10分，表現相對「較好」。這或許暗示學生在該科目（或其相關領域）存在一定程度的理解基礎或學習潛力，儘管其絕對分數仍處於極低的範疇，遠未達到基本及格標準。

*   **弱項領域（亟需